In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#定义数据预处理
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize([32, 32]),
     #将原始输入数据resize为32*32
     transforms.Normalize((0.1307,), (0.3081,))]
)

In [3]:
data_dir = '/Users/sunxiaolei/Python_Projects/Deeplearning/experiment_four/datasets'
#加载数据集
trainset = MNIST(root=data_dir, train = True,download=True, transform = transform)
testset = MNIST(root=data_dir, train = False, download=True, transform = transform)
#定义数据加载器
trainloader = DataLoader(trainset, batch_size = 64, shuffle = True, num_workers=2)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

In [4]:
#定义LeNet模型
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1) #卷积层1
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2) # 池化层1
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool1(x)
        x = torch.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 16 * 5 * 5) # reshape
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
# 初始化模型和优化器
model = LeNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001) # 使用Adam优化器

In [6]:
model

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [7]:
# training model
losses = [] # record train loss
accuracy = [] # record precision
test_losses = [] # record test loss
test_accuracy = [] # record test precision
num_epochs = 10

for epoch in range(num_epochs):
    training_loss = 0.0
    training_corrects = 0.0
    testing_loss = 0.0
    testing_corrects = 0.0

    # train
    for i, (inputs, labels) in enumerate(trainloader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, predicted = torch.max(outputs, 1)
        loss.backward()
        optimizer.step()
        training_loss += loss.item() * inputs.size(0)
        training_corrects += torch.sum(predicted == labels.data)

    # test
    # There is no need to turn on automatic derivation and backpropagation when testing
    with torch.no_grad():
        # 将模型转换为评估模式，在该模模式下不会影响到训练
        model.eval()
        for i, (inputs, labels) in enumerate(testloader):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs, 1)
            testing_loss += loss.item() * inputs.size(0)
            testing_corrects += torch.sum(predicted == labels.data)

    model.train()
    training_loss = training_loss / len(trainset)
    training_corrects = training_corrects / len(trainset)
    testing_loss = testing_loss / len(testset)
    testing_corrects = testing_corrects / len(testset)
    losses.append(training_loss)
    accuracy.append(training_corrects)
    test_losses.append(testing_loss)
    test_accuracy.append(testing_corrects)
    print("epochs: {}/{}:\n".format(epoch+1, num_epochs),
          "Train_loss: {:.3f}".format(training_loss),
          "Test_loss: {:.3f}".format(testing_loss),
          "Train_acc: {:.3f}".format(training_corrects),
          "Test_acc: {:.3f}".format(testing_corrects))



/Users/sunxiaolei/miniconda3/envs/pytorch/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/Users/sunxiaolei/miniconda3/envs/pytorch/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change fro

epochs: 1/10:
 Train_loss: 0.269 Test_loss: 0.082 Train_acc: 0.917 Test_acc: 0.974


/Users/sunxiaolei/miniconda3/envs/pytorch/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/Users/sunxiaolei/miniconda3/envs/pytorch/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change fro

epochs: 2/10:
 Train_loss: 0.077 Test_loss: 0.053 Train_acc: 0.976 Test_acc: 0.983


/Users/sunxiaolei/miniconda3/envs/pytorch/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/Users/sunxiaolei/miniconda3/envs/pytorch/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change fro

KeyboardInterrupt: 

In [ ]:
# # 将GPU tensor 转为cpu tensor
# for i in range(len(accuracy)):
#     accuracy[i] = accuracy[i].cpu()
# for i in range(len(test_accuracys)):
#     test_accuracys[i] = test_accuracys[i].cpu()

# loss chart
plt.subplot(1, 2, 1)
plt.plot(losses, label = 'Training loss')
plt.plot(test_losses, label = 'Testing loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss')
plt.legend()

# precision chart
plt.subplot(1, 2, 2)
plt.plot(accuracy, label = 'Training Accuracy')
plt.plot(test_accuracy, label = 'Testing Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy')
plt.legend()
plt.show()